In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as p

## Define couples derivatives to integrate

In [6]:
def dydx(x,y):
    
    #set the derivatives
    
    #our equation is d^2y/dx^2 = -y
    
    #so we can write
    #dydx = z
    #dzdx = -y
    
    y_derivs = np.zeros(2)
    
    y_derivs[0] = y[1]
    
    y_derivs[1] = -1*[0]
    
    return y_derivs

## Define the 4th order RK method

In [7]:
def rk4_mc_core(dydx,xi,yi,nv,h):
    
    #declare k? arrays
    k1 = np.zeros(nv)
    k2 = np.zeros(nv)
    k3 = np.zeros(nv)
    k4 = np.zeros(nv)
    
    #define x at 1/2 step
    x_ipoh = xi + 0.5*h
    
    #define x at 1 step
    x_ipoh = xi + h
    
    #declare a temp y array
    y_temp = np.zeros(nv)
    
    #get k1 values
    y_derivs = dyx(xi,yi)
    k1[:] = h*y_derivs[:]
    
    #get k2 values
    y_temp[:] = yi[:] + 0.5*k1[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k2[:] = h*y_derivs[:]
    
    #get k3 values:
    y_temp[:] = yi[:] + 0.5*k2[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k3[:] = h*y_derivs[:]
    
    #get k4 values
    y_temp[:] = yi[:] + k3[:]
    y_derivs = dydx(x_ipoh,y_temp)
    k4[:] = h*y_derivs[:]
    
    #adv y by a step h
    yipo = yi + (k1 + 2*k2 + 2*k3 + k4)/6.
    
    return yipo

## Define an adaptive step size driver for RK4

In [8]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    #set a max number of iterations
    imax = 10000
    
    #set an interation variable 
    i = 0
    
    #crate an error
    Delta = np.full(nv,2*tol)
    
    #remember the step
    h_step = h
    
    #adjust step
    while(Delta.max()/tol > 1.0):
        
        #estimate our error by talking one step size h
        #vs. two steps of size h/2
        y_2  = rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1  = rk4_mv_core(dydx,x_i,y_i,nv,0.5*h_step)
        y_11 = rk4_mv_core(dydx,x_i+0.5*h_step,y_1,nv,0.5*h_step)
        
        #compute an error
        Deltae = np.fabs(y_2 = y_11)
        
        #if the error is too large, take a smaller step
        if(Delta.max()/tol > 1.0):
            
            #our error is too lrage, decrease the step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
        
        #check iteration
        if (i>=imax):
            print("Too many interations in rk4_mv_ad()")
            raise StopIteration("Ending after i - ",i)
            
        #iterate
        i+1
        
        #next time, try to take a bigger step
        h_new = np.fmin(H_step * (Delta.max()/tol)**(-0.9), h_step*H_NEW_FAC)
        
        #return the answer, a new step, and the step we actually took
        return y_2, h_new, h_step

## Define wrapper for RK4

In [ ]:
def rk4_mv(dfdx,a,b,y_a,tol):
    
    #define our sarting step
    xi = a
    yi = y_a.copy()
    
    #an intial step size == make very small!
    h = 1.0e-4 * (b-a)
    
    #set a max num of iterations
    imax = 10000
    
    #set an iteration variable
    i = 0
    
    #set  the number of couples odes to the size of y_a
    nv = (y_a)
    
    #set the initial conditions
    x = np.full(l,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag = 1
    
    3loop until we reach the right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        #update the step
        h = h_new
        
        #prevent an overshoot
        if(xi+h_step>b):
            
            #take a smaller step
            h = b-xi
            
            yi_new, h_new, h_step = rk4_mv_ad(dydx,xi,yi,nv,h,tol)
            
            flag = 0
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        #add the step to the arrays
        x = np.append(x,xi)
        y_new = np.zeros((lens(x),nv))
        y_new[0:len(x)-1,:] = y
        y_new[-1,:] = yi[:]